Парсер Zoon.ru.
Собирает данные со страниц объекта:

- телефон
- часы работы
- ссылку на вебсайт и аккаунты в соц.сетях
- тип клиники
- услуги
- рейтинг клиники
- количество отзывов
- описание
- "официальные" данные
- количество загруженных сертификатов
- инфо о специалистах (имя, стаж, специализация)
- краткий прайс

Полный прайс — в другом ноутбуке.
Парсер написан 2021-08-05, доработан 2021-08-10

Надо доработать выгрузку спецов

In [1]:
#Загружаю нужные библиотеки

import pandas as pd

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument('--proxy-server=??.??.???.???:?????') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [5]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)

In [14]:
# выгружаю ссылки из готового датасета

work_links = pd.read_excel('C:/Data/Стоматологии/data_stom.xlsx', sheet_name='data')

In [15]:
work_links.sample()

,clinic_page,clinic,address,region,city,phone,hours,website,social_network,fb_link,...,clinic_rating,review_number,description,clinic_info,doc_pics,specialists_dict,price,services,contacts,soc_links
663,https://zoon.ru/msk/medical/stomatologiya_auru...,Aurum-dent,"м. Селигерская, Бескудниковский бульвар, 30к3,...",NaN,NaN,84994891790,ежедневно: 09:00 - 20:00,none,Instagram,none,...,3.7,25.0,Новая стоматологическая клиника Аурум Дент отк...,['Главный врач: Максим Леонидович Акрамов'],0.0,"{'Максим Леонидович Акрамов ': ['Главный врач,...","{'Лечение кариеса': '1500—3000', 'Удаление зуб...",{'Тип мед. учреждения': 'стоматологическая кли...,"{'Время работы': ['ежедневно: 09:00 - 20:00'],...",none


In [18]:
# отрезаю пару лишних строк (из-за особенностей датасета)

work_links = work_links[4081:]

In [19]:
work_links

,clinic_page,clinic,address,region,city,phone,hours,website,social_network,fb_link,...,clinic_rating,review_number,description,clinic_info,doc_pics,specialists_dict,price,services,contacts,soc_links
4081,https://zoon.ru/msk/medical/mnogoprofilnyj_med...,Президент,"м. Коломенская, Якорная улица, 7к1, Москва",Москва,Москва,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4082,https://zoon.ru/msk/medical/stomatologiya_novo...,НовоДент,"м. Прокшино, Москва, поселение бульвар Веласке...",Москва,Москва,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
data = [] # список для данных

for i in tqdm(range(work_links.shape[0])):
    driver.get(work_links.iloc[i].clinic_page)
    
    page_url = work_links.iloc[i].clinic_page
    object_name = work_links.iloc[i].clinic # название и адрес для идентификации
    address = work_links.iloc[i].address

    try:
        clinic_rating = driver.find_element_by_class_name('rating-value').text # рейтинг

    except:
        clinic_rating = 0

    try:
        review_number = driver.find_element_by_class_name('fs-large.gray.js-toggle-content').text # кол-во отзывов

    except:
        review_number = 0
        
    # телефон
    try:
        phone = driver.find_element_by_class_name('service-phones-list').find_element_by_tag_name('a').get_attribute('href')

    except:
        phone = 'none'
        
    

    # часы работы
    try:
        hours = driver.find_element_by_class_name('upper-first').text

    except:
        hours = 'none'

    # описание клиники
    try:
        description = driver.find_element_by_class_name('js-desc.oh.word-break').text

    except:
        description = 'none'

    # из-за особенностей верстки вытаскиваю блоки текста и разбиваю на нужные элементы
    service_blocks = driver.find_element_by_class_name('service-description-block').find_elements_by_class_name('fluid') # левый блок с услугами

    # вытаскиваю описание услуг в словарь
    services = {}
    for i in service_blocks:
        temp_list = i.text.split('\n')
        services[temp_list[0]] = temp_list[1]

    # нужные данные
    try:
        clinic_type = services['Тип мед. учреждения']
        
    except:
        clinic_type = 'none'
    
    try:
        stom_services = services['Стоматология']
        
    except:
        stom_services = 'none'

    # блок с контактами
    info_blocks = driver.find_element_by_class_name('service-maininfo').find_element_by_class_name('js-placecycle').find_elements_by_class_name('fluid')

    # вытаскиваю контакты в словарь
    contacts = {}

    for info in info_blocks:
        temp_list = info.text.split('\n')

        try:        
            contacts[temp_list[0]] = temp_list[1:]

        except:
            contacts[info.text] = 'no_text'

        # разбиваю ссылки на соц.сети
        if temp_list[0] == 'Страница в соцсетях':
            social_links = info.find_elements_by_tag_name('a')

            soc_links = {} # вытаскиваю ссылки на соц.сети в словарь
            for soc_link in social_links:
                soc_links[soc_link.text] = soc_link.get_attribute('href')

            try:
                fb_link = soc_links['Facebook']
            except:
                fb_link = 'none'

            try:
                inst_link = soc_links['Instagram']
            except:
                inst_link = 'none'

            try:
                vk_link = soc_links['ВКонтакте']

            except:
                vk_link = 'none'

            try:
                tw_link = soc_links['Твиттер']

            except:
                tw_link = 'none'
                
        else:
            fb_link, inst_link, vk_link, tw_link, soc_links = 'none', 'none', 'none', 'none', 'none'
            

    # сайт
    try:
        website = contacts['Официальный сайт'] or contacts['сайт']

    except:
        website = 'none'

    # количество сертификатов
    try:
        doc_pics = len(driver.find_element_by_class_name('js-prof-certs-photos').find_elements_by_class_name('js-photo-wrapper'))

    except:
        doc_pics = 0  

    # страницы в соц.сетях

    try:
        social_network = contacts['Страница в соцсетях']

    except:
        social_network = 'none'


    # официальная инфо
    try:
        clinic_info = contacts['Информация']

    except:
        clinic_info = 'none'

    # раскрываю всех специалистов

    while True:

        try:
            button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'button.button-show-more'))
            )
            button.click()

        except:
            break

    # выгружаю данные о специализации и стаже 

    specialists_dict = {}
    
    try:
        specialists = driver.find_element_by_class_name('prof-list').find_elements_by_tag_name('li') # блоки со специалистами
        
        for spec_block in specialists:
            name = spec_block.find_element_by_tag_name('a').text
            prof = spec_block.find_element_by_class_name('prof-item__orientation').text.replace(' •', ';')
            specialists_dict[name] = prof.split(';')
            
    except:
        pass
            
    # избранные цены
    
    price = {}
    
    try:
        prices = driver.find_element_by_class_name('mp-markerlist').find_elements_by_class_name('mp-markerlist-item')

        for item in prices:
            price_item = item.text.split('\n')
            price[price_item[0]] = price_item[1]
            
    except:
        pass
        
    
    # добавляю данные в список
    data.append([page_url, object_name, address, phone, hours, website,
                 social_network, fb_link, inst_link, vk_link, tw_link,
                 clinic_type, stom_services, clinic_rating, review_number,
                 description, clinic_info, doc_pics, specialists_dict, price,            
                services, contacts, soc_links])

  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
# записываю данные в датафрейм

head_list = ['page_url', 'object_name', 'address', 'phone', 'hours', 'website',
                 'social_network', 'fb_link', 'inst_link', 'vk_link', 'tw_link',
                 'clinic_type', 'stom_services', 'clinic_rating', 'review_number',
                 'description', 'clinic_info', 'doc_pics', 'specialists_dict', 'price',            
                'services', 'contacts', 'soc_links']
work = pd.DataFrame(data, columns=head_list)

In [22]:
# Записываю в файл

work.to_excel('C:/Data/stom_info_1.xlsx', encoding='utf-8', index=False)

In [ ]:
driver.quit()